In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 2000)
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
data_path = '/home/mvkrein/etf_model/data'
etf_data_file = os.path.join(data_path,'etf_new_var_20180910.csv')
etf_data = pd.read_csv(etf_data_file,index_col=0)

In [4]:
etf_study_file = os.path.join(data_path,'ETF_list_min_6yr_history.csv')
etf_list = pd.read_csv(etf_study_file,index_col=0)

In [5]:
etf_predict_file = os.path.join(data_path,'etf_pred_random_forest_regress_20181008.csv')
etf_predict = pd.read_csv(etf_predict_file,index_col=0)

In [6]:
etf_predict.head(5)

,Date,sym,rank_p_L-21,predict
118881,2015-02-05,AAXJ,0.274510,0.496871
118882,2015-02-05,ACWI,0.601307,0.619799
118883,2015-02-05,ACWV,0.379085,0.451211
118884,2015-02-05,ACWX,0.509804,0.512270
118885,2015-02-05,AGG,0.320261,0.495049


In [7]:
# etf_predict.columns=['Date_pred','sym_pred','rank_p_L-21_pred','predict']

In [8]:
etf_data.sort_values(['Date','sym'],ascending=True,inplace=True)
etf_data.reset_index(drop=True,inplace=True)
etf_predict['rank_predict'] = (etf_predict.groupby('Date')['predict'].rank(method='dense', ascending=False))
etf_predict.head(5)

,Date,sym,rank_p_L-21,predict,rank_predict
118881,2015-02-05,AAXJ,0.274510,0.496871,93.0
118882,2015-02-05,ACWI,0.601307,0.619799,25.0
118883,2015-02-05,ACWV,0.379085,0.451211,115.0
118884,2015-02-05,ACWX,0.509804,0.512270,84.0
118885,2015-02-05,AGG,0.320261,0.495049,94.0


In [9]:
dates = list(etf_data['Date'].unique())

In [10]:
output_df = (etf_data.loc[:,['Date','sym','delta_p_L-21','target']]).merge\
    (etf_predict.loc[:,['Date','sym','predict','rank_predict']],left_on=['Date','sym'],right_on=['Date','sym'],how='inner')

In [11]:
output_df.tail(5)

,Date,sym,delta_p_L-21,target,predict,rank_predict
6421,2018-07-09,TUR,-0.129697,0,0.461883,98.0
6422,2018-07-09,USIG,0.002842,0,0.619746,17.0
6423,2018-07-09,USMV,0.027387,1,0.585543,40.0
6424,2018-07-09,USRT,0.007042,0,0.390863,144.0
6425,2018-07-09,WPS,-0.013406,0,0.546212,54.0


In [12]:
assess_df = output_df.loc[((output_df['rank_predict']<=5.0) & (output_df['Date']<'2018-07-26')),:]
return_assess = pd.DataFrame(assess_df.groupby('Date')['delta_p_L-21'].mean())
return_assess.columns = ['top_5_avg_rtn']
return_assess.reset_index(drop=False,inplace=True)
return_assess

,Date,top_5_avg_rtn
0,2015-02-05,-0.045547
1,2015-03-09,0.061600
2,2015-04-08,0.002820
3,2015-05-07,-0.004315
4,2015-06-08,-0.129629
5,2015-07-08,0.015440
6,2015-08-06,-0.075340
7,2015-09-04,-0.001510
8,2015-10-06,0.013309
9,2015-11-04,-0.060522


In [13]:
return_assess_vs_mkt = (etf_data.loc[etf_data['sym']=='IVV',['Date','sym','delta_p_L-21']]).merge\
(return_assess,left_on=['Date'],right_on=['Date'],how='inner')
return_assess_vs_mkt

,Date,sym,delta_p_L-21,top_5_avg_rtn
0,2015-02-05,IVV,0.010741,-0.045547
1,2015-03-09,IVV,0.002660,0.061600
2,2015-04-08,IVV,0.004202,0.002820
3,2015-05-07,IVV,-0.001902,-0.004315
4,2015-06-08,IVV,-0.013770,-0.129629
5,2015-07-08,IVV,0.018605,0.015440
6,2015-08-06,IVV,-0.076446,-0.075340
7,2015-09-04,IVV,0.032631,-0.001510
8,2015-10-06,IVV,0.063911,0.013309
9,2015-11-04,IVV,-0.003687,-0.060522


In [14]:
def monthly_sharpe(returns, N=12):

# Calculate the annualised Sharpe ratio of a returns stream 
# based on a number of trading periods, N. N defaults to 252,
# which then assumes a stream of daily returns.

# The function assumes that the returns are the excess of 
# those compared to a benchmark.

    return np.sqrt(N) * returns.mean() / returns.std()

In [15]:
monthly_sharpe(return_assess_vs_mkt['delta_p_L-21'])

1.1506812817790062

In [16]:
monthly_sharpe(return_assess_vs_mkt['top_5_avg_rtn'])

0.4440177059065338

In [17]:
cum_rtn = np.column_stack((np.cumprod((1+return_assess_vs_mkt['delta_p_L-21'])),np.cumprod((1+return_assess_vs_mkt['top_5_avg_rtn']))))
cum_rtn

array([[1.01074074, 0.95445343],
       [1.01342893, 1.01324733],
       [1.01768741, 1.01610482],
       [1.0157519 , 1.01172015],
       [1.00176471, 0.88057146],
       [1.02040244, 0.89416729],
       [0.94239646, 0.82680092],
       [0.97314799, 0.82555218],
       [1.03534302, 0.83653911],
       [1.03152607, 0.7859102 ],
       [0.98405906, 0.7503026 ],
       [0.93026774, 0.6819214 ],
       [0.98213799, 0.72812862],
       [1.01516483, 0.7212592 ],
       [1.02427781, 0.76913102],
       [1.05379622, 0.80561668],
       [1.04832616, 0.84855333],
       [1.09181985, 0.91648435],
       [1.09664695, 0.91780357],
       [1.08480948, 0.93821606],
       [1.0504481 , 0.92621552],
       [1.11131822, 0.94715777],
       [1.14559988, 0.98272366],
       [1.15887979, 0.99477789],
       [1.19766351, 1.006504  ],
       [1.19689979, 1.03838981],
       [1.21847331, 1.08698111],
       [1.23908688, 1.09521013],
       [1.2369227 , 1.09069693],
       [1.26641057, 1.14357589],
       [1.

In [18]:
def cagr(cum_rtns):
    yrs = len(cum_rtns)/12
    cagr = cum_rtns[len(cum_rtns)-1]**(1/yrs) - 1
    return cagr

In [19]:
# cagr(cum_rtn[:,1]) ridge regression alpha 1.0 17.2% 1.02 sharpe  alpha 0.8 = 17.68%  1.07 sharpe

In [20]:
cagr(cum_rtn[:,1])

0.05684572014101352

In [21]:
cagr(cum_rtn[:,0])

0.12003636878397539